In [1]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista = ['BTCUSDT','ETHUSDT','BCHUSDT','XRPUSDT','LTCUSDT','LINKUSDT','ADAUSDT','BNBUSDT','ZRXUSDT','DOGEUSDT','MKRUSDT','DOTUSDT','TRBUSDT','RUNEUSDT','SOLUSDT','STORJUSDT','AVAXUSDT','ENJUSDT','FILUSDT','MATICUSDT','COTIUSDT','1000SHIBUSDT','BAKEUSDT','DYDXUSDT','GALAUSDT','APEUSDT','OPUSDT','LQTYUSDT','ARBUSDT','LEVERUSDT','1000PEPEUSDT','WLDUSDT','BNTUSDT','ARKUSDT']
#lista = ['ETHUSDT','LTCUSDT','ADAUSDT','DOTUSDT','FILUSDT','COTIUSDT'] #ELEGIDOS
#lista = ['LQTYUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
print(lista)
for symbol in lista:    
    try:
        #print(symbol)
        data = md.estrategia_royal(symbol, debug = False, refinado = False, file_source = False, timeframe = '1h')
        resultado=md.backtesting_royal(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#(data.loc[data['Open Time'] == '2023-10-03 03:00:00'])


['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ZRXUSDT', 'DOGEUSDT', 'MKRUSDT', 'DOTUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'STORJUSDT', 'AVAXUSDT', 'ENJUSDT', 'FILUSDT', 'MATICUSDT', 'COTIUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'DYDXUSDT', 'GALAUSDT', 'APEUSDT', 'OPUSDT', 'LQTYUSDT', 'ARBUSDT', 'LEVERUSDT', '1000PEPEUSDT', 'WLDUSDT', 'BNTUSDT', 'ARKUSDT']
BTCUSDT - Return [%]: 3.31% - # Trades: 4 - Profit Factor: 3.590485253648365 - Win Rate [%]: 50.0
ETHUSDT - Return [%]: -1.98% - # Trades: 4 - Profit Factor: 0.0 - Win Rate [%]: 0.0
BCHUSDT - Return [%]: -3.67% - # Trades: 10 - Profit Factor: 0.5399681515731845 - Win Rate [%]: 10.0
XRPUSDT - Return [%]: -2.57% - # Trades: 10 - Profit Factor: 0.7052759577088721 - Win Rate [%]: 10.0
LTCUSDT - Return [%]: -3.07% - # Trades: 4 - Profit Factor: 0.0 - Win Rate [%]: 0.0
LINKUSDT - Return [%]: 6.1% - # Trades: 2 - Profit Factor: 9.022169869966215 - Win Rate [%]: 50.0
ADAUSDT - Return [%]: 1.08% - # Tra